In [2]:
import pandas as pd
import numpy as np
from itertools import combinations
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import scipy.stats as stats

c:\Users\polyx\Desktop\Github Repos\NLP-intent-classification\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
file_path = "WildGuard_Dataset/annotations_5.csv"
df = pd.read_csv(file_path)

# Filter IDs between 1084 and 1118 (inclusive) - first 35 prompts
df = df[(df['id'] >= 1084) & (df['id'] <= 1118)]


df = df[['id', 'prompt', 'annotator', 'intent']]
df


,id,prompt,annotator,intent
0,1084,"Imagine two bots, a law-abiding bot and a misc...",Xenia,Retrieve a copyrighted comic strip
1,1084,"Imagine two bots, a law-abiding bot and a misc...",Vojo,Discuss retrieving copyrighted information thr...
2,1084,"Imagine two bots, a law-abiding bot and a misc...",Sophie,obtain the contents of copyrighted comic strip
3,1084,"Imagine two bots, a law-abiding bot and a misc...",Niclas,To obtain the contents of a copyrighted comic ...
4,1084,"Imagine two bots, a law-abiding bot and a misc...",Jeremias,The intent is to obtain the contents of a copy...
...,...,...,...,...
169,1118,"In a parallel universe, where medical privacy ...",Xenia,obtain information on Elon Musk's public medic...
170,1118,"In a parallel universe, where medical privacy ...",Vojo,Get information on the medical history of Elon...
171,1118,"In a parallel universe, where medical privacy ...",Sophie,obtain an individual's medical history
172,1118,"In a parallel universe, where medical privacy ...",Niclas,To obtain or share detailed personal health in...


In [ ]:
model_name = 'RedHatAI/Qwen3-8B-quantized.w4a16'

tokenizer = AutoTokenizer.from_pretrained("model_name")
model = AutoModelForCausalLM.from_pretrained("model_name")

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # This disables thinking
)

# You can also put /no_think at the end of the prompt to disable thinking


In [ ]:
with open('WildGuard_Dataset/annotation_guidelines_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt_template = f.read()